In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional
from keras.models import load_model
from keras import callbacks, regularizers, optimizers
from keras.regularizers import L1L2
from keras_tuner import RandomSearch, Objective
import scipy
import feature_engine
import pmdarima
from statsmodels.tsa.ar_model import AutoReg
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error

AR, ARIMA

In [ ]:
data = pd.read_csv('./data/preprocessed_data.csv', index_col=0, parse_dates=True, squeeze=True)
test_size = 52
data_train, data_val, data_test = data[ :-2*test_size], data[-2*test_size:-test_size], data[-test_size:]
data_train_val, data_test = data[ :-test_size], data[-test_size:]

In [ ]:
train_X = data_train.values
val_X  = data_val.values
test_X = data_test.values
train_val_X = data_train_val.values

In [ ]:
AM_model = AutoReg(train_val_X, lags=13)
AM_model_fit = AM_model.fit()
AM_predictions = AM_model_fit.predict(start=len(train_val_X), end=len(train_val_X)+len(test_X)-1, dynamic=False)
AM_predictions = pd.Series(AM_predictions, index=data_test.index)

In [ ]:
final_predictions = pd.DataFrame(data_test)
final_differences = pd.DataFrame(data_test)
def add_pred(series, name):
    final_predictions[name] = series
    return final_predictions
def cal_diff(series, name):
    final_differences[name+"_diff"] = data_test - series
    return final_differences

In [ ]:
add_pred(AM_predictions, 'AM_predictions')
cal_diff(AM_predictions, 'AM_predictions')

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

best_cfg = (13, 1, 0)
best_ARIMA = ARIMA(train_val_X, order = best_cfg)
best_ARIMA_fit = best_ARIMA.fit()
best_ARIMA_predictions = pd.Series(best_ARIMA_fit.predict(start = len(train_val_X), end=len(train_val_X)+ len(test_X)-1, dynamic = False))
best_ARIMA_predictions = best_ARIMA_predictions.set_axis(data_test.index)

In [ ]:
add_pred(best_ARIMA_predictions, 'best_ARIMA_predictions')
cal_diff(best_ARIMA_predictions, 'best_ARIMA_predictions')

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

best_cfg = np.array(['(3, 1, 2)', '(3, 0, 0, 52)'], dtype='<U16')
best_SARIMA = SARIMAX(endog = train_val_X, order = eval(best_cfg[0]), seasonal_order = eval(best_cfg[1]))
best_SARIMA_fit = best_SARIMA.fit()
best_SARIMA_predictions = pd.Series(best_SARIMA_fit.predict(start = len(train_val_X), end=len(train_val_X)+ len(test_X)-1, dynamic = False))
best_SARIMA_predictions = best_SARIMA_predictions.set_axis(data_test.index)

In [ ]:
add_pred(best_SARIMA_predictions, 'sarima1')
cal_diff(best_SARIMA_predictions, 'sarima1')

In [ ]:
best_cfg = np.array(['(2, 1, 1)', '(2, 0, 0, 52)'], dtype='<U16')
best_SARIMA2 = SARIMAX(endog = train_val_X, order = eval(best_cfg[0]), seasonal_order = eval(best_cfg[1]))
best_SARIMA_fit2 = best_SARIMA2.fit()
best_SARIMA_predictions2 = pd.Series(best_SARIMA_fit2.predict(start = len(train_val_X), end=len(train_val_X)+ len(test_X)-1, dynamic = False))
best_SARIMA_predictions2 = best_SARIMA_predictions2.set_axis(data_test.index)

In [ ]:
add_pred(best_SARIMA_predictions2, 'sarima2')
cal_diff(best_SARIMA_predictions2, 'sarima2')

# LSTM

In [ ]:
# the data after log transformation
series = pd.read_csv('./data/preprocessed_data.csv', index_col=0, parse_dates=True, squeeze=True)
data = pd.DataFrame(series.values, index=range(0,312))
data.columns = ['bookings']

test_size = 52
data_train, data_test =  data[ :-test_size], data[-test_size:]


from keras.preprocessing.sequence import TimeseriesGenerator

def data_generator(data, n_input, n_features):
    generator = TimeseriesGenerator(data = data, targets= data, length=n_input, batch_size=1)
    return generator

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

scaler.fit(data_train)
scaled_train_data = scaler.transform(data_train)
scaled_test_data = scaler.transform(data_test)
scaled_data = np.append(scaled_train_data, scaled_test_data)

d_train = data_generator(scaled_train_data, 13, 1)
d_test = data_generator(np.append(scaled_train_data[-52:], scaled_test_data), 13, 1)

def predict_lstm(n_input, model, test_data):
    lstm_predictions_scaled = list()

    for i in range(len(test_data)): 
        batch = scaled_data[-52-n_input:-1][i:i+n_input]
        current_batch = batch.reshape((1, n_input, 1))  
        lstm_pred = model.predict(current_batch, verbose=0)[0]
        lstm_predictions_scaled.append(lstm_pred) 
        current_batch = np.append(current_batch[:,1:,:],[[lstm_pred]],axis=1)
    lstm_predictions = scaler.inverse_transform(lstm_predictions_scaled)
    lstm_predictions = pd.Series(lstm_predictions.reshape(1,52)[0], index=data_test.index)
    return lstm_predictions

In [ ]:
def cal_diff_lstm(series, name):
    final_differences[name+"_diff"] =  data_test.to_numpy().reshape(1,-1)[0] - np.array(series)
    return final_differences

In [ ]:
uni_lstm = load_model('./model/univariate_stack1.h5')
uni_lstm_prediction = predict_lstm(13, uni_lstm, data_test)
uni_lstm_prediction = np.array(uni_lstm_prediction)

In [ ]:
add_pred(uni_lstm_prediction, 'uni_lstm')
cal_diff_lstm(uni_lstm_prediction, 'uni_lstm')

In [ ]:
uni_bi_lstm = load_model('./model/bidirectional_r')
uni_bi_lstm_prediction = predict_lstm(13, uni_bi_lstm, data_test)
uni_bi_lstm_prediction = np.array(uni_bi_lstm_prediction)

In [ ]:
add_pred(uni_bi_lstm_prediction, 'uni_bi_lstm')
cal_diff_lstm(uni_bi_lstm_prediction, 'uni_bi_lstm')

## multivariate LSTM

In [ ]:
area_sum = pd.read_csv('./data/area sum.csv', index_col=0, parse_dates=True).squeeze("columns")

test_size = 52
data_train, data_test =  area_sum.iloc[ :-test_size,:], area_sum.iloc[-test_size:,:]

# log transformation
data_train, data_test = np.log(data_train+1), np.log(data_test+1)

# split the dataset for normalization
train_X = data_train.iloc[:,:-1]
train_y = np.array(data_train.iloc[:,-1]).reshape(-1,1)
test_X = data_test.iloc[:,:-1]
test_y = np.array(data_test.iloc[:,-1]).reshape(-1,1)

# normalization

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

scaler_X.fit(train_X)
scaled_train_X = scaler_X.transform(train_X)
scaled_test_X = scaler_X.transform(test_X)

scaler_y.fit(train_y)
scaled_train_y = scaler_y.transform(train_y)
scaled_test_y = scaler_y.transform(test_y)

scaled_X = np.append(scaled_train_X, scaled_test_X, axis=0)

print(scaled_train_X.shape, scaled_test_X.shape, scaled_X.shape)

In [ ]:
# walking forward validation
def predict_lstm(n_input, n_feature, model, data_test, data):
    lstm_predictions_scaled = list()

    for i in range(data_test.shape[0]): 
        batch = data[-52-n_input:-1, :][i:i+n_input, :]
        current_batch = batch.reshape((1, n_input, n_feature))
        lstm_pred = model.predict(current_batch, verbose=0)[0]
        lstm_predictions_scaled.append(lstm_pred)
    lstm_predictions = scaler_y.inverse_transform(lstm_predictions_scaled)
    lstm_predictions = pd.Series(lstm_predictions.reshape(1,52)[0], index=data_test.index)
    return lstm_predictions

In [ ]:
multi_lstm = load_model('./model/vanilla_area.h5')
multi_lstm_prediction = predict_lstm(13, 9, multi_lstm, data_test, scaled_X)
multi_lstm_prediction = np.array(multi_lstm_prediction)

In [ ]:
def cal_diff_lstm_multi(series, name):
    final_differences[name+"_diff"] =  data_test['All'].to_numpy().reshape(1,-1)[0] - np.array(series)
    return final_differences

In [ ]:
add_pred(multi_lstm_prediction, 'multi_lstm')
cal_diff_lstm_multi(multi_lstm_prediction, 'multi_lstm')

In [ ]:
multi_bi_lstm = load_model('./model/bidirectional_area')
multi_bi_lstm_prediction = predict_lstm(13, 9, multi_bi_lstm, data_test, scaled_X)
multi_bi_lstm_prediction = np.array(multi_bi_lstm_prediction)

In [ ]:
add_pred(multi_bi_lstm_prediction, 'multi__bi_lstm')
cal_diff_lstm_multi(multi_bi_lstm_prediction, 'multi_bi_lstm')

In [ ]:
final_difference2 = final_differences.iloc[:,1:]
final_difference2.columns = ['AM', 'ARIMA', 'SARIMA1', 'SARIMA2', 'Univariate LSTM', 'Univariate BI-LSTM', 'Multivariate LSTM', 'Multivariate BI-LSTM']

In [ ]:
plt.figure(figsize = (10, 3))
plt.plot(final_difference2, lw=1.5)
plt.legend(final_difference2.columns, bbox_to_anchor=(1, 1.05))
plt.xticks(np.arange(260, 312, step=2))
plt.axhline(y=0, color='purple', ls='--', lw=1)
plt.vlines([260, 273, 286, 299, 311], ymin=final_difference2.min().min(), ymax= final_difference2.max().max(), colors='blue', ls='--', lw=1)
plt.show()

In [ ]:
plt.figure(figsize = (10, 3))
plt.plot(final_difference2.iloc[:,[0,1]], lw=1.5)
plt.legend(['AM','ARIMA'])
plt.xticks(np.arange(260, 312, step=2))
plt.axhline(y=0, color='purple', ls='--', lw=1)
plt.vlines([260, 273, 286, 299, 311], ymin=final_difference2.min().min(), ymax= final_difference2.max().max(), colors='blue', ls='--', lw=1)
plt.show()

In [ ]:
plt.figure(figsize = (10, 3))
plt.plot(final_difference2.iloc[:,[1,2,3]], lw=1.5)
plt.legend(['ARIMA','SARIMA1', 'SARIMA2'])
plt.xticks(np.arange(260, 312, step=2))
plt.axhline(y=0, color='purple', ls='--', lw=1)
plt.vlines([260, 273, 286, 299, 311], ymin=final_difference2.min().min(), ymax= final_difference2.max().max(), colors='blue', ls='--', lw=1)
plt.show()

In [ ]:
plt.figure(figsize = (10, 3))
plt.plot(final_difference2.iloc[:,[4,5]], lw=1.5)
plt.legend(['Univariate LSTM', 'Univariate BI-LSTM'])
plt.xticks(np.arange(260, 312, step=2))
plt.axhline(y=0, color='purple', ls='--', lw=1)
plt.vlines([260, 273, 286, 299, 311], ymin=final_difference2.min().min(), ymax= final_difference2.max().max(), colors='blue', ls='--', lw=1)
plt.show()

In [ ]:
plt.figure(figsize = (10, 3))
plt.plot(final_difference2.iloc[:,[6,7]], lw=1.5)
plt.legend(['Multivariate LSTM', 'Multivariate BI-LSTM'])
plt.xticks(np.arange(260, 312, step=2))
plt.axhline(y=0, color='purple', ls='--', lw=1)
plt.vlines([260, 273, 286, 299, 311], ymin=final_difference2.min().min(), ymax= final_difference2.max().max(), colors='blue', ls='--', lw=1)
plt.show()

In [ ]:
plt.figure(figsize = (10, 3))
plt.plot(final_difference2.iloc[:,[2,5,7]], lw=1.5)
plt.legend(['SARIMA1', 'Univariate LSTM', 'Multivariate BI-LSTM'])
plt.xticks(np.arange(260, 312, step=2))
plt.axhline(y=0, color='purple', ls='--', lw=1)
plt.vlines([260, 273, 286, 299, 311], ymin=final_difference2.min().min(), ymax= final_difference2.max().max(), colors='blue', ls='--', lw=1)
plt.show()